In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely

In [2]:
# Spacing in kilometers
def create_grid(x_min, x_max, y_min, y_max, spacing):
    polygons, centers = [], []
    width = spacing/(111)
    
    for x in np.linspace(x_min, x_max, round((x_max-x_min)/width)):
        for y in np.linspace(y_min, y_max, round((y_max-y_min)/width)):
            poly = shapely.geometry.Polygon([(x,y),(x+width,y),(x+width,y+width),(x,y+width)])
            center = poly.centroid
            polygons.append(poly)
            centers.append(center)
            
    res = pd.DataFrame({'ID':list(range(len(polygons))),
                        'Polygon':polygons,
                        'Center':centers})
    res = gpd.GeoDataFrame(res, geometry=res['Polygon'], crs='EPSG:4326')
    return res


### Generate Grid

In [9]:
# Read in data, using crs EPSG:4326
df = create_grid(121.1,121.5,14.5,14.8,0.500)
df = gpd.GeoDataFrame(df, geometry=df['Center'], crs='EPSG:4326')

# Keep only points within Antipolo
mun = gpd.read_file("muni/MuniCities.shp", crs='EPSG:4326')
ant_mun = mun.loc[mun.NAME_2.str.contains('Antipolo')]
ant_mun = gpd.GeoDataFrame(ant_mun, geometry=ant_mun['geometry'], crs='EPSG:4326')

df = gpd.sjoin(df, ant_mun, how='left', op='within')
df = df.loc[df.ISO.notnull()].reset_index(drop=True)[['ID','Polygon']]

# Reset ID
df = df.drop('ID', axis=1)
df['Grid_ID'] = [str(x) for x in list(range(1,df.shape[0]+1))]

In [10]:
# Output
pd.DataFrame(df).to_csv('antipolo_grid.csv', index=False)